In [57]:
import relay_bp
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


import sinter
import multiprocessing

import sys
from pathlib import Path

# Make test utilities importable
tests_dir = Path.cwd().parent / "tests"
if str(tests_dir) not in sys.path:
    sys.path.insert(0, str(tests_dir))

# Make BBcode module importable (from qPLDC-circuits/tomas)
# bb_dir = (Path.cwd().parents[2] / "qPLDC-circuits" / "tomas").resolve()
# if str(bb_dir) not in sys.path:
#     sys.path.insert(0, str(bb_dir))
import BBcode

# Import Relay-BP Sinter decoder API
from relay_bp.stim.sinter.decoders import (
    SinterDecoder_RelayBP,
    sinter_decoders,
)

In [58]:
import os, sys, importlib
from pathlib import Path

qasm_dir = "./bb_code_qasm"
os.makedirs(qasm_dir, exist_ok=True)

# New: directory for saved results
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

# Add stabilizer/src to PYTHONPATH using a path relative to this notebook
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path.cwd().resolve()
src_dir = (base_dir.parents[3] / "src").resolve()  # .../stabilizer/src
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

import noise_util as ns
importlib.reload(ns)

<module 'noise_util' from '/hpc/home/garn195/NWQ-Sim/stabilizer/src/noise_util.py'>

In [59]:
import subprocess
import time

# Compile the C++ code once before the loop
print("Compiling C++ simulator (MPI)...")
# Adjust the path to your C++ file as needed
cpp_file_path = './bb_code_sim.cpp'
executable_path = './bb_code_sim'

compile_command = [
    "mpicxx",  # Use MPI compiler wrapper
    "-std=c++17",
    "-O3",
    "-I../../../../../", # Include for NWQ-Sim headers
    "-DMPI_ENABLED",
    "-o",
    executable_path,
    cpp_file_path
]
try:
    subprocess.run(compile_command, check=True, capture_output=True, text=True)
    print("Compilation successful.")
except subprocess.CalledProcessError as e:
    print("Compilation failed.")
    print("--- stdout ---")
    print(e.stdout)
    print("--- stderr ---")
    print(e.stderr)

Compiling C++ simulator (MPI)...
Compilation successful.


In [60]:

#Params
circuit = "bicycle_bivariate_144_12_12_memory_Z"
distance = 12
rounds = 12
SHOTS = 100000

XYZ_decoding = False

# Construct Relay-BP decoder (from decoders.py)
decoder = SinterDecoder_RelayBP(
    gamma0=0.1,
    pre_iter=80,
    num_sets=300,
    set_max_iter=60,
    gamma_dist_interval=[-0.24, 0.66],
    stop_nconv=1,
    parallel=True
)

# circuit = get_test_circuit(circuit=circuit, distance=distance, rounds=rounds, error_rate=error_rate)
# if not XYZ_decoding:
#     circuit = filter_detectors_by_basis(circuit, "Z")

# circuit.diagram("timeline-svg")


In [61]:
#Parameters
tasks = []
T1 = 10 ** -4
T2 = 10 ** -4
tau = np.array([10**-6, 2*10**-6, 4*10**-6, 6*10**-6, 8*10**-6, 10**-5])


lam = 1/T2 - 1/(2*T1)
p_amp = 1 - np.exp(-tau/T1)
p_phase = 1 - np.exp(-lam*tau)
p_list = (p_amp, p_phase)

# gate_lam = 1/T2 - 1/(2*T1)
# gate_p_amp = 1 - np.exp(-gate_tau/T1)
# gate_p_phase = 1 - np.exp(-lam*gate_tau)
# gate_p_list = (gate_p_amp, gate_p_phase)
base_error = 10**-3

bb_code = BBcode.BBcode(
    n=144, k=12, d=12, m=6, l=12,
    A=[[3, 0], [0, 1], [0, 2]],
    B=[[0, 3], [1, 0], [2, 0]],
    shift=[0, 0],
    f=[[0, 0], [1, 0], [2, 0], [3, 0], [6, 0], [7, 0], [8, 0], [9, 0], [1, 3], [5, 3], [7, 3], [11, 3]],
    g=[[1, 0], [2, 1], [0, 2], [1, 2], [2, 3], [0, 4]],
    h=[[0, 0], [0, 1], [1, 1], [0, 2], [0, 3], [1, 3]],
    alpha=[[0, 0], [0, 1], [2, 1], [2, 5], [3, 2], [4, 0]],
    beta=[[0, 1], [0, 5], [1, 1], [0, 0], [4, 0], [5, 2]],
)
print(p_amp)
print(p_phase)
i=0
for p_amp, p_phase in zip(p_amp, p_phase):
    noise_profile = [0,0,0,0]
    bb_circuit = bb_code.build_full_BBcode_circuit(rounds=rounds, noise_profile=noise_profile, observable_type="Z", code_capacity=True)

    #Generate stim circuit
    model = ns.ErrorModel(bb_circuit)
    model.setting_error('Identity', False, f'DEPOLARIZE1({base_error})')
    # model.setting_error('Single_qubit', True, f'PAULI_CHANNEL_1({gate_p_amp/4}, {gate_p_amp/4}, {(.5-gate_p_amp/4-np.sqrt(1-gate_p_amp-gate_p_phase))/2})')
    # model.setting_error('Two_qubit', False, f'PAULI_CHANNEL_1({gate_p_amp/4}, {gate_p_amp/4}, {(.5-gate_p_amp/4-np.sqrt(1-gate_p_amp-gate_p_phase))/2})')
    model.setting_error('Single_qubit', False, f'DEPOLARIZE1({base_error})')
    model.setting_error('Two_qubit', False, f'DEPOLARIZE2({base_error})')
    model.setting_error('Measurement', True, f'DEPOLARIZE1({p_amp/2+p_phase/4})')#f'PAULI_CHANNEL_1({p_amp/4}, {p_amp/4}, {((1-np.sqrt((1-p_amp)*(1-p_phase)))/2)-p_amp/4})')
    model.setting_error('Reset', False, f'DEPOLARIZE1({p_amp/2+p_phase/4})') #'PAULI_CHANNEL_1({p_amp/4}, {p_amp/4}, {((1-np.sqrt((1-p_amp)*(1-p_phase)))/2)-p_amp/4})')
    stim_circuit = model.generate_noisy_circuit()
    tasks.append(sinter.Task(circuit=stim_circuit, json_metadata={'d': distance, "trial":i}))

    #Generate stabsim circuit
    # model.setting_error('Single_qubit', True, f'DEPOLARIZE1({base_error})')
    # model.setting_error('Two_qubit', True, f'DEPOLARIZE2({base_error})')
    model.setting_error('Measurement', True, f'AMPLITUDE_DAMP({p_phase}, {p_amp})')
    model.setting_error('Reset', False, f'AMPLITUDE_DAMP({p_phase}, {p_amp})')
    stab_circuit = model.generate_noisy_circuit()
    qasm_output = ns.stim_to_qasm_with_depolarize_noise(stab_circuit)
    # Inject AMPLITUDE_DAMP around M/RESET in QASM using the model settings
    qasm_output = ns.inject_amplitude_damp(qasm_output, model)
    
    try:
        qasm_path = os.path.join(qasm_dir, f"bb_code_d{distance}_p{i}.qasm")
        with open(qasm_path, "w") as f:
            f.write(qasm_output)
    except Exception as e:
        print(f"Failed to export QASM for p={p_amp}: {e}")
    i+=1

print(tau)

[0.00995017 0.01980133 0.03921056 0.05823547 0.07688365 0.09516258]
[0.00498752 0.00995017 0.01980133 0.02955447 0.03921056 0.04877058]
[1.e-06 2.e-06 4.e-06 6.e-06 8.e-06 1.e-05]


In [62]:
# stim_circuit.diagram(type="timeline-svg")

In [63]:
print(tau)

[1.e-06 2.e-06 4.e-06 6.e-06 8.e-06 1.e-05]


In [64]:
def run_stim_experiment(tasks, shots):
    stim_lers = []
    if not tasks:
        return stim_lers

    for i, task in enumerate(tasks):
        print(f"\n--- Running Stim for task = {i} ---")
        start_time = time.perf_counter()
        
        # Use sinter.collect for parallel decoding
        collected_stats = sinter.collect(
            num_workers=os.cpu_count() // 2,
            tasks=[task],
            decoders=['relay_bp'],
            max_shots=shots,
            custom_decoders={
                'relay_bp': decoder
            },
            print_progress=True,
        )
        
        stim_time = time.perf_counter() - start_time
        
        if collected_stats:
            stat = collected_stats[0]
            stim_ler = float(stat.errors) / float(stat.shots) if stat.shots > 0 else 0.0
        else:
            stim_ler = 0.0

        stim_lers.append(stim_ler)
        print(f"Stim LER: {stim_ler}, Sim+Decoding Time: {stim_time:.4f}s\n")

    return stim_lers


# --- Run Stim-only experiment ---

stim_logical_error_rates = run_stim_experiment(tasks, SHOTS)
print("\nStim complete.")
print("Probabilities:", p_list)
print("Stim LERs:", stim_logical_error_rates)

# Save Stim results
try:
    stim_results_path = os.path.join(results_dir, f"stim_results_d{distance}_shots{SHOTS}_dep.txt")
    with open(stim_results_path, "w") as f:
        f.write("# tau\tp_amp\tp_phase\tstim_ler\n")
        for ta, pa, pp, ler in zip(tau, p_amp, p_phase, stim_logical_error_rates):
            f.write(f"{ta}\t{pa}\t{pp}\t{ler}\n")
    print(f"Saved Stim results to {stim_results_path}")
except Exception as e:
    print("Failed to save Stim results:", e)


Starting 224 workers...



--- Running Stim for task = 0 ---


1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99999           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99807           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99686           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99250           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      98454           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      97610           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      96889           0 d=12,trial=0 
1 tasks left:
  workers  decoder eta shots_left errors_seen js

Stim LER: 0.00044, Sim+Decoding Time: 112.2544s


--- Running Stim for task = 1 ---


1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99999           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99975           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99935           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99869           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99759           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99293           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      98812           0 d=12,trial=1 
1 tasks left:
  workers  decoder eta shots_left errors_seen js

Stim LER: 0.00094, Sim+Decoding Time: 152.4129s


--- Running Stim for task = 2 ---


1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99999           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99886           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99845           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99737           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      99424           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      98634           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen json_metadata
      224 relay_bp <1m      98268           0 d=12,trial=2 
1 tasks left:
  workers  decoder eta shots_left errors_seen js

KeyboardInterrupt: 

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import math
import relay_bp_mp # Import the new helper module

env = os.environ.copy()
env.update({
    "OMP_NUM_THREADS": "1",
    "OPENBLAS_NUM_THREADS": "1",
    "MKL_NUM_THREADS": "1",
    "VECLIB_MAXIMUM_THREADS": "1",
    "NUMEXPR_NUM_THREADS": "1",
})

def run_cpp_experiment(tasks, shots):
    cpp_lers = []
    if not tasks:
        return cpp_lers

    for task in tasks:
        trial = task.json_metadata.get("trial")
        d = distance

        ref_circ = task.circuit
        m2d_converter = ref_circ.compile_m2d_converter()
        dem = ref_circ.detector_error_model(
            decompose_errors=True,
            ignore_decomposition_failures=True,
        )
        compiled_decoder = decoder.compile_decoder_for_dem(dem=dem)
        
        print(f"\n--- Running C++ (MPI) for trial {trial} ---")

        # Paths (absolute to avoid cwd confusion)
        qasm_file_path = os.path.abspath(os.path.join(qasm_dir, f"bb_code_d{d}_p{trial}.qasm"))
        cpp_output_path = os.path.abspath(os.path.join(qasm_dir, f"measurements_d{d}_p{trial}.bin"))

        # Remove any stale output
        if os.path.exists(cpp_output_path):
            os.remove(cpp_output_path)

        # MPI execution
        num_qubits = task.circuit.num_qubits
        mpi_ranks = max(1, min(os.cpu_count()//2, shots))
        run_command = [
            "mpirun", "-np", str(mpi_ranks),
            "./bb_code_sim", str(num_qubits), str(shots), qasm_file_path, cpp_output_path
        ]

        cpp_time = 0.0
        try:
            start_time = time.perf_counter()
            result = subprocess.run(run_command, check=True, capture_output=True, text=True, env=env)
            cpp_time = time.perf_counter() - start_time
            # Optional: print stdout if needed for debugging
            # print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"C++ simulation failed for trial {trial}.")
            print(e.stderr)
            cpp_lers.append(0.0)
            continue

        # --- Parallel Decoding with ProcessPoolExecutor ---
        cpp_ler = 0.0
        decode_start_time = time.perf_counter()
        try:
            # 1) Load packed binary measurements
            with open(cpp_output_path, "rb") as f:
                header_u32 = np.fromfile(f, dtype=np.uint32, count=2)
                header_u64 = np.fromfile(f, dtype=np.uint64, count=2)
                magic, version = int(header_u32[0]), int(header_u32[1])
                mlen, shots_file = int(header_u64[0]), int(header_u64[1])
                if magic != 0x4E575142: raise RuntimeError("Invalid magic")
                bytes_per_shot = (mlen + 7) // 8
                packed = np.fromfile(f, dtype=np.uint8)

            if shots_file == 0 or packed.size == 0:
                cpp_lers.append(0.0); continue

            bit_packed_meas = np.ascontiguousarray(packed.reshape(shots_file, bytes_per_shot))

            # 2) Convert measurements -> detection events + observables
            det_samples, obs_flips = m2d_converter.convert_measurements_bit_packed(
                bit_packed_meas, separate_observables=True, bit_packed_order='little'
            )

            # 3) Parallel decode using ProcessPoolExecutor
            total_shots = len(det_samples)
            if total_shots > 0:
                num_decode_workers = max(1, os.cpu_count() // 2)
                chunk_size = math.ceil(total_shots / num_decode_workers)
                
                det_chunks = [det_samples[i:i + chunk_size] for i in range(0, total_shots, chunk_size)]
                obs_chunks = [obs_flips[i:i + chunk_size] for i in range(0, total_shots, chunk_size)]

                with ProcessPoolExecutor(
                    max_workers=num_decode_workers,
                    initializer=relay_bp_mp.worker_init,
                    initargs=(compiled_decoder, dem.observables_matrix),
                ) as executor:
                    total_errors = sum(executor.map(relay_bp_mp.count_errors, det_chunks, obs_chunks))

                cpp_ler = float(total_errors) / float(total_shots)
            
            stab_decode_time = time.perf_counter() - decode_start_time
            print(f"C++ LER: {cpp_ler:.4f}, Sim Time: {cpp_time:.4f}s, Decoding Time {stab_decode_time:.4f}s")

        except Exception as e:
            print(f"C++ decoding failed for trial {trial}: {e}")
            cpp_ler = 1.0

        cpp_lers.append(cpp_ler)

    return cpp_lers

# --- Run C++-only experiment ---
cpp_logical_error_rates = run_cpp_experiment(tasks, SHOTS)
print("\nC++ complete.")
print("Probabilities:", p_list)
print("C++ LERs:", cpp_logical_error_rates)

# Save C++ results
try:
    p_amp_arr = 1 - np.exp(-tau / T1)
    lam = 1 / T2 - 1 / (2 * T1)
    p_phase_arr = 1 - np.exp(-lam * tau)
    cpp_results_path = os.path.join(results_dir, f"cpp_results_d{distance}_shots{SHOTS}.txt")
    with open(cpp_results_path, "w") as f:
        f.write("# tau\tp_amp\tp_phase\tcpp_ler\n")
        for ta, pa, pp, ler in zip(tau, p_amp_arr, p_phase_arr, cpp_logical_error_rates):
            f.write(f"{ta}\t{pa}\t{pp}\t{ler}\n")
    print(f"Saved C++ results to {cpp_results_path}")
except Exception as e:
    print("Failed to save C++ results:", e)


In [ ]:
# --- Plotting Results ---
plt.figure(figsize=(6, 10))

plt.plot(tau/T1, stim_logical_error_rates, 'o-', label='Stim LER (Relay BP)')
plt.plot(tau/T1, cpp_logical_error_rates, 's--', label='C++ LER (Relay BP)')
plt.xlabel(r"$\tau / T1$ Ratio")
plt.ylabel("Logical Error Rate (LER)")
plt.title(rf"LER M Damping BB Code (rounds=d={distance}), $T1,T2 = {T1*10**6},{T2*10**6} \mu s$, Shots={SHOTS}")
plt.xscale('log')
plt.yscale('log')
plt.xlim()
# plt.xticks(p_list_extend, [f"{p:.0e}" for p in p_list_extend])
plt.grid(True, which="both", ls="--")
plt.legend()
plt.tight_layout()
plt.savefig(f'../../../figures/BB_code_LER{SHOTS}.pdf', dpi=500, bbox_inches='tight')
plt.show()

# New cell: Load from files and re-plot without recomputing


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Reload and plot previously saved results
SHOTS = 1000000
try:
    # Fallback if results_dir isn't defined in this session
    results_dir = globals().get('results_dir', './results')
    stim_results_path = Path(results_dir) / f"stim_results_d{distance}_shots{SHOTS}.txt"
    cpp_results_path = Path(results_dir) / f"cpp_results_d{distance}_shots{SHOTS}.txt"

    if not stim_results_path.exists() or not cpp_results_path.exists():
        missing = []
        if not stim_results_path.exists():
            missing.append(str(stim_results_path))
        if not cpp_results_path.exists():
            missing.append(str(cpp_results_path))
        raise FileNotFoundError("Missing results files: " + ", ".join(missing))

    def load_results(path):
        data = []
        with open(path, 'r') as f:
            for line in f:
                if line.startswith('#') or not line.strip():
                    continue
                parts = line.strip().split('\t')
                if len(parts) >= 4:
                    ta, pa, pp, ler = map(float, parts[:4])
                    data.append((ta, pa, pp, ler))
        data = np.array(data)
        return data[:, 0], data[:, 1], data[:, 2], data[:, 3]

    tau_s, p_amp_s, p_phase_s, stim_ler_s = load_results(stim_results_path)
    tau_c, p_amp_c, p_phase_c, cpp_ler_c = load_results(cpp_results_path)

    # Basic consistency check
    # if not np.allclose(tau_s, tau_c):
    #     print("Warning: tau arrays differ between Stim and C++ results; proceeding with Stim taus for x-axis.")

    plt.figure(figsize=(6, 10))
    plt.plot(tau_s / T1, stim_ler_s, 'o-', label='Stim LER (Relay BP) [loaded]')
    plt.plot(tau_c / T1, cpp_ler_c, 's--', label='C++ LER (Relay BP) [loaded]')
    plt.xlabel(r"$\tau / T1$ Ratio")
    plt.ylabel("Logical Error Rate (LER)")
    plt.title(rf"Loaded LER M Damping BB Code (rounds=d={distance}), $T1,T2 = {T1*10**6},{T2*10**6} \mu s$, Shots={SHOTS}")
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True, which="both", ls="--")
    plt.legend()
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Failed to load and plot saved results:", e)

Failed to load and plot saved results: name 'distance' is not defined
